Astromatch
==========

Astromatch is a python module for statistical cross-matching of astronomical catalogues. It implements different matching methods and provides a consistent output format (as far as possible) for all of them. This allows easy comparisons between different methods.

Astromatch is design as a coherent framework, well integrated with other tools in the Astropy package. When using Astromatch, you should keep in mind that it is more a library that allows you to write your own python scripts for cross-matching astronomical catalogues, than a simple stand-alone software.

Catalogue objects
---------------------------

Astromatch provides a useful tool for defining an astronomical catalogue. The basic information needed for building a ``Catalogue`` object is the position of the sources, the positional errors, their ID labels and the area covered by the catalogue. Having a reasonable estimation of the catalogue area is crucial when using statistical cross-matching methods.

As en example, lets build an Astromatch Catalogue for X-ray sources in a sky region of 1 deg radius within the XXL-North survey. 

Lets firs retrieve the data we need from the Vizier database (we will use only the data from the 0.5-2 keV band):

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table


region_center = SkyCoord(ra=35.30109, dec=-4.45962, unit='deg')
search_radius = 1*u.deg

try:
    xcat_table = Table.read('xxl_1deg.fits')

except FileNotFoundError:
    from astroquery.vizier import Vizier

    v = Vizier(
        columns=['Xseq', 'RABdeg', 'DEBdeg', 'e_Bpos'],
        column_filters={'e_Bpos':'>0', 'Bp1': '==1'},
        row_limit=-1,
    )
    result = v.query_region(region_center, radius=search_radius, catalog='IX/52/3xlss')
    xcat_table = result[0]
    xcat_table.meta['description'] = ''
    xcat_table.write('xxl_1deg.fits', format='fits', overwrite=True)

We use the output of the Vizier query for building the corresponding Astromatch catalogue:

In [ ]:
import numpy as np
from astromatch import Catalogue

xcat = Catalogue(
    xcat_table,
    name='xxl',
    id_col='Xseq',
    coord_cols=['RABdeg', 'DEBdeg'],
    poserr_cols=['e_Bpos'],
    poserr_type='circle',
    area=np.pi * search_radius**2,
)
xcat

Astropy units for coordinates and positional errors of the Catalogue are always defined. If the initial data have no units defined for those columns, Astromatch asigns by default degrees to the coordinates and arcsec for the positional errors. By default it assumes that coordinates are in the ICRS reference frame (equatorial system). Other frames can be used by setting the `frame` parameter while defining the Catalogue (see the Astropy SkyCoord documentation for all suported frames). Internally all coordinates frames are transformed into ICRS.

Instead of passing an Astropy Table for the definition of the catalogue, a path to a file (in a tabular format suported by Astropy) can be used:

In [ ]:
xcat = Catalogue(
    'xxl_1deg.fits',
    name='xxl',
    id_col='Xseq',
    coord_cols=['RABdeg', 'DEBdeg'],
    poserr_cols=['e_Bpos'],
    poserr_type='circle',
    area=np.pi * search_radius**2,
)

The area of the Catalogue can be set using a MOC object (or the path to a file containing the MOC). The ``mocpy`` package have multiple tools for defining MOCs. For example, for the circular sky region we queried, we can define the following MOC:

In [ ]:
from mocpy import MOC

moc_xxl = MOC.from_elliptical_cone(
    lon=region_center.ra,
    lat=region_center.dec,
    a=search_radius,
    b=search_radius,
    pa=0*u.deg,
    max_depth=14
)

Now we can use the MOC for the Catalogue definition:

In [ ]:
xcat = Catalogue(
    'xxl_1deg.fits',
    name='xxl',
    id_col='Xseq',
    coord_cols=['RABdeg', 'DEBdeg'],
    poserr_cols=['e_Bpos'],
    poserr_type='circle',
    area=moc_xxl,
)
xcat.area

We strongly recomend using MOCs for defining the areas covered by Catalogues. When cross-matching catalogues with partial common coverage, Astromatch can use the corresponding MOCs to cross-match only the sources in the common area.

Catalogues can be access by index and sliced as Python lists:

In [ ]:
xcat[10]
xcat[10:20]
xcat[[10, 20, 30]]

It is also possible to select sources by their ids:

In [ ]:
list_of_ids = ['225336', '225330', '225412']
xcat_new = xcat.select_by_id(list_of_ids)
xcat_new

Sources can also be removed by id from a Catalogue:

In [ ]:
list_of_ids = ['225336', '225330', '225412']
xcat_new = xcat.remove_by_id(list_of_ids)
xcat_new

Two Catalogue objects can be joined in a single Catalogue:

In [ ]:
cat1 = xcat[1:4]
cat2 = xcat[10:14]

cat1.join(cat2)

The different atributes of the Catalogue can be access independently:

In [ ]:
# IDs
xcat.ids

# Area
xcat.area

# Coordinates
xcat.coords

# Positional errors
xcat.poserr

The positional errors are defined as SkyCoordErr objects. This class allows to manipulate the errors in several ways. For example, adding a systematic error:

In [ ]:
# Save the original positional errors
old_poserr = xcat.poserr[:]

# Add systematic error to the catalogue
xcat.poserr.add_syserr(1.5*u.arcsec)
print(xcat)

# Recover original errors
xcat.poserr = old_poserr

Cross-matching: Match
----------------------------------

The ``Match`` class is the basic tool for cross-matching Catalogues. It can use several matching algorithms, and implements different methods to calculate the statistical properties (completeness, error rate, etc) of the results.

As an example, we will cross-match the catalogue of X-ray sources we defined above with a catalogue of optical sources from the SDSS. First, we download the data we need, if it is not already available. We will use the optical magnitudes during the cross-match, so we need to download that too:

In [ ]:
sdss_mags = ['umag', 'gmag', 'rmag', 'imag', 'zmag']

try:
    ocat_table = Table.read('sdss_1deg.fits')

except FileNotFoundError:
    from astroquery.vizier import Vizier

    sdss_cols = ['objID', 'RA_ICRS', 'DE_ICRS', 'e_RA_ICRS', 'e_DE_ICRS'] + sdss_mags

    v = Vizier(columns=sdss_cols,
               column_filters={'mode': '=1', 'q_mode': '=+', 'e_RA_ICRS': '>0', 'e_DE_ICRS': '>0'},
               row_limit=-1,
    )
    result = v.query_region(region_center, radius=1*u.deg, catalog='V/147/sdss12')
    ocat_table = result[0]
    ocat_table.meta['description'] = ''
    ocat_table.write('sdss_1deg.fits', format='fits', overwrite=True)

Now we can define a new Catalogue. Since the optical sources are from the exact same sky region as the X-ray sources, we can use the MOC we defined above to set the area of the Catalogue. Note also that here we are using a different type for the positional errors:

In [ ]:
ocat = Catalogue(
    ocat_table,
    name='sdss',
    id_col='objID',
    coord_cols=['RA_ICRS', 'DE_ICRS'],
    poserr_cols=['e_RA_ICRS', 'e_DE_ICRS'],
    poserr_type='rcd_dec_ellipse',
    area=moc_xxl,
    mag_cols=sdss_mags,
)

We define a match between `xcat` and `ocat`:

In [ ]:
from astromatch import Match

xm = Match(xcat, ocat)

We calculate the cross-match using the likelihood ratio method:

In [ ]:
match_results_lr = xm.run(method='lr', radius=10.0*u.arcsec)

By default all cross-matching methods included in astromatch try to estimate magnitude priors using the information provided within the catalogues.

In [ ]:
xm.priors.plot('rmag')
print("Overal identification rate in r band:", xm.priors.qcap('rmag'))


Priors can be stored into files for later use:

In [ ]:
prior_table = xm.priors.to_table(include_bkg_priors=True)
prior_table.write("lrpriors.fits", format="fits", overwrite=True)


Full output of the LR algorithm:

In [ ]:
xm.lr

All matches within the search radius (10 arcsec):

In [ ]:
all_matches = xm.get_matchs()
all_matches

List of most likely counterparts:

In [ ]:
primary_matches = xm.get_matchs(match_type='primary')
primary_matches

List of most likely counterparts, including X-ray sources with no counterparts:

In [ ]:
primary_matches = xm.get_matchs(match_type='primary_all')
primary_matches

Set as best matches only counterparts with LR above a given threshold:

In [ ]:
xm.set_best_matchs(cutoff=0.5)

best_matches = xm.get_matchs(match_type='best')
best_matches

If no LR threshold is given, the method will try to find the optimum threshold (maximizing the completeness while minimizing the error rate):

In [ ]:
xm.set_best_matchs()

best_matches = xm.get_matchs(match_type='best')
best_matches

DRA, DDEC offsets between matches:

In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import norm

dra, ddec = xm.offset('xxl', 'sdss', match_type='best')
x = np.linspace(-10, 10, num=100)

fig = plt.figure(figsize=(13, 5))

plt.subplot(121)
plt.hist(dra, bins='auto', density=True)
plt.plot(x, norm.pdf(x, np.mean(dra), np.std(dra)))
plt.xlabel('DRA / arcsec')

plt.subplot(122)
plt.hist(ddec, bins='auto', density=True)
plt.plot(x, norm.pdf(x, np.mean(ddec), np.std(ddec)))
plt.xlabel('DDEC / arcsec')

plt.show()

How to obtain the information in the optical and X-ray catalogues for the primary counterparts:

In [ ]:
ocat_best = ocat.select_by_id(best_matches['SRCID_sdss'])
ocat_best

In [ ]:
xcat_best = xcat.select_by_id(best_matches['SRCID_xxl'])
xcat_best

Join X-ray and optical information of the best matches in a single Astropy Table:

In [ ]:
from astropy.table import hstack

best_all = hstack(
    [xcat_best.save(), ocat_best.save()],  # Catalogues can be converted into Tables using the `save` method
    table_names=[xcat_best.name, ocat_best.name],
    join_type='exact')
best_all

There are differents methods to characterize the statistical properties of the cross-match.

Using the probabilities estimated by the LR algorithm:

In [ ]:
lr_stats = xm.stats(ncutoff=501, maxcutoff=10.0)



By cross-matching with a randomized catalogue of X-ray sources:

In [ ]:
lr_stats_rnd = xm.stats(match_rnd=True, ncutoff=501, maxcutoff=10.0)


By using the randomization method described in Broos et al. 2006:

In [ ]:
lr_stats_broos = xm.stats(use_broos=True, ntest=1, ncutoff=501, maxcutoff=10.0)

In [ ]:
fig = plt.figure(figsize=(13, 5))

plt.subplot(121)
plt.plot(lr_stats['cutoff'], lr_stats['error_rate'], lw=3)
plt.plot(lr_stats_rnd['cutoff'], lr_stats_rnd['error_rate'], lw=3, ls=':')
plt.plot(lr_stats_broos['cutoff'], lr_stats_broos['error_rate'], lw=3, ls='--')

plt.xlabel('LR')
plt.ylabel('error rate')


plt.subplot(122)
plt.plot(lr_stats['reliability'], lr_stats['completeness'], lw=3)
plt.plot(lr_stats_rnd['reliability'], lr_stats_rnd['completeness'], lw=3, ls=':')
plt.plot(lr_stats_broos['reliability'], lr_stats_broos['completeness'], lw=3, ls='--')

plt.xlabel('purity')
plt.ylabel('completeness')

plt.tight_layout()
plt.show()

We repeat the cross-matching of our two catalogues using NWAY:

In [ ]:
match_nway_nomags = xm.run(method='nway', radius=10.0*u.arcsec, prior_completeness=0.55)

The match we have just performed does not take into account the optical magnitudes of the SDSS sources. If we want to use that information during the cross-matching, we set the `use_mags` parameter to `True` and astromatch automatically estimates magnitude priors from the photometric data contained in the optical catalogue:

In [ ]:
match_nway_mags_default = xm.run(method='nway', radius=10.0*u.arcsec, use_mags=True, prior_completeness=0.55)

By default NWAY uses high likelihood counterparts (matches with high posterior probability estimated using only the positional information) to calculate the magnitude priors. Alternatively, you can build priors using the same method implemented in the LR algorithm:

In [ ]:
match_nway_mags_custom = xm.run(
    method='nway', radius=10.0*u.arcsec, prior_completeness=0.55,
    use_mags=True,
    bayes_prior=False,
# Default settings for LR magnitude priors
    mag_include_radius=10*u.arcsec,
    magmin=10.0,
    magmax=30.0,
    magbinsize=0.5,
    rndcat=True,
)

Or we can just use the magnitude prior we calculated during the LR match. In this case we need to define a dictionary containing the priors for each secondary catalogue that is included in the match (only the SDSS catalogue in this particular example):

In [ ]:
from astromatch.priors import Prior

priors = {'sdss': Prior.from_table("lrpriors.fits", sdss_mags)}

match_nway_mags_custom2 = xm.run(
    method='nway', radius=10.0*u.arcsec, prior_completeness=0.55, use_mags=True, priors=priors
)

In [ ]:
fig = plt.figure(figsize=(4, 5))

plt.hist(match_nway_nomags['prob_has_match'], bins='auto')
plt.hist(match_nway_mags_default['prob_has_match'], bins='auto', histtype='step', lw=3)
plt.hist(match_nway_mags_custom['prob_has_match'], bins='auto', histtype='step', lw=3, ls='--')
plt.hist(match_nway_mags_custom2['prob_has_match'], bins='auto', histtype='step', lw=3, ls=':')

plt.show()

DRA, DDEC offsets between matches for NWAY results:

In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import norm

xm.set_best_matchs(cutoff=0.8)
dra, ddec = xm.offset('xxl', 'sdss', match_type='best')
x = np.linspace(-10, 10, num=100)

fig = plt.figure(figsize=(13, 5))

plt.subplot(121)
plt.hist(dra, bins='auto', density=True)
plt.plot(x, norm.pdf(x, np.mean(dra), np.std(dra)))
plt.xlabel('DRA / arcsec')

plt.subplot(122)
plt.hist(ddec, bins='auto', density=True)
plt.plot(x, norm.pdf(x, np.mean(ddec), np.std(ddec)))
plt.xlabel('DDEC / arcsec')

plt.show()

In [ ]:
nway_stats = xm.stats(ncutoff=100, maxcutoff=0.99)
nway_stats_rnd = xm.stats(match_rnd=True, ncutoff=100, maxcutoff=0.99, prior_completeness=0.55)
nway_stats_broos = xm.stats(use_broos=True, ntest=5, ncutoff=100, maxcutoff=0.99, prior_completeness=0.55)

In [ ]:
fig = plt.figure(figsize=(13, 5))

plt.subplot(121)
plt.plot(nway_stats['cutoff'], nway_stats['error_rate'], lw=3)
plt.plot(nway_stats_rnd['cutoff'], nway_stats_rnd['error_rate'], lw=3, ls=':')
plt.plot(nway_stats_broos['cutoff'], nway_stats_broos['error_rate'], lw=3, ls='--')

plt.xlabel('prob_has_match')
plt.ylabel('error rate')


plt.subplot(122)
plt.plot(nway_stats['reliability'], nway_stats['completeness'], lw=3)
plt.plot(nway_stats_rnd['reliability'], nway_stats_rnd['completeness'], lw=3, ls=':')
plt.plot(nway_stats_broos['reliability'], nway_stats_broos['completeness'], lw=3, ls='--')

plt.xlabel('purity')
plt.ylabel('completeness')

plt.tight_layout()
plt.show()